# to get start
- source .venv/bin/activate or use conda env
- jupyter notebook --port 8899 --ip 0.0.0.0 [in new terminal, not in vscode]

[recommand]
- also, we can you conda env . then select env upper right.
- In MPK, use ```conda activate openpi-droid```

### install pyzed
https://www.stereolabs.com/docs/development/python/install

(1) install ZED SDK

(2)
```cd "/usr/local/zed/" python3 get_python_api.py```



# pi0 deployment details
- wrist camera & its mount
- normalize gripper state
- image resilution: In droid: (180, 320, 3). 

### select location

In [ ]:
# LOCATION = 'MPK'
LOCATION = 'FRE'

In [ ]:
import dataclasses

import jax

from openpi.models import model as _model
from openpi.policies import droid_policy
from openpi.policies import policy_config as _policy_config
from openpi.shared import download
from openpi.training import config as _config
from openpi.training import data_loader as _data_loader

import Pyro5.api
import numpy as np

# add roboarena checkpoints

```
### Trained from PaliGemma, using RT-2 / OpenVLA style binning tokenizer.
uv run scripts/serve_policy.py policy:checkpoint --policy.config=paligemma_binning_droid --policy.dir=gs://openpi-assets/checkpoints/roboarena/paligemma_binning_droid

### Trained from PaliGemma, using FAST tokenizer (using universal FAST+ tokenizer).
uv run scripts/serve_policy.py policy:checkpoint --policy.config=paligemma_fast_droid --policy.dir=gs://openpi-assets/checkpoints/roboarena/paligemma_fast_droid

### Trained from PaliGemma, using FAST tokenizer (tokenizer trained on DROID dataset).
uv run scripts/serve_policy.py policy:checkpoint --policy.config=paligemma_fast_specialist_droid --policy.dir=gs://openpi-assets/checkpoints/roboarena/paligemma_fast_specialist_droid

### Trained from PaliGemma, using FSQ tokenizer.
uv run scripts/serve_policy.py policy:checkpoint --policy.config=paligemma_vq_droid --policy.dir=gs://openpi-assets/checkpoints/roboarena/paligemma_vq_droid

### pi0-style diffusion / flow VLA, trained on DROID from PaliGemma.
uv run scripts/serve_policy.py policy:checkpoint --policy.config=paligemma_diffusion_droid --policy.dir=gs://openpi-assets/checkpoints/roboarena/paligemma_diffusion_droid
```

# load model

### pi0_droid

In [ ]:
config = _config.get_config("pi0_droid")
checkpoint_dir = download.maybe_download("gs://openpi-assets/checkpoints/pi0_droid")

# Create a trained policy.
policy = _policy_config.create_trained_policy(config, checkpoint_dir)



### pi0_FAST_droid

In [ ]:
config = _config.get_config("pi0_fast_droid")
checkpoint_dir = download.maybe_download("gs://openpi-assets/checkpoints/pi0_fast_droid")

# Create a trained policy.
policy = _policy_config.create_trained_policy(config, checkpoint_dir)

### paligemma_binning_droid

In [ ]:
config = _config.get_config("paligemma_binning_droid")
checkpoint_dir = download.maybe_download("gs://openpi-assets/checkpoints/roboarena/paligemma_binning_droid")

# Create a trained policy.
policy = _policy_config.create_trained_policy(config, checkpoint_dir)

### paligemma_diffusion_droid

In [ ]:
config = _config.get_config("paligemma_diffusion_droid")
checkpoint_dir = download.maybe_download("gs://openpi-assets/checkpoints/roboarena/paligemma_diffusion_droid")

# Create a trained policy.
policy = _policy_config.create_trained_policy(config, checkpoint_dir)

### run it if you want to switch policy

In [ ]:
# delete current policy to free up memory, then load a new one
del policy

### sanity check

In [ ]:
# Run inference on a dummy example. This example corresponds to observations produced by the DROID runtime.
example = droid_policy.make_droid_example()
result = policy.infer(example)
print("Actions shape:", result["actions"].shape)

# initialize realsense external cameras

In [ ]:
import pyrealsense2 as rs
import numpy as np
import cv2
import matplotlib.pyplot as plt



class RealSenseCamera:
    def __init__(self, serial_number, width=320, height=180, fps=30):
        self.serial = serial_number
        self.pipeline = rs.pipeline()
        self.config = rs.config()
        self.config.enable_device(self.serial)
        self.config.enable_stream(rs.stream.color, width, height, rs.format.bgr8, fps)
        self.pipeline.start(self.config)




    def get_image(self):
        frames = self.pipeline.wait_for_frames()
        color_frame = frames.get_color_frame()
        if not color_frame:
            return None

        bgr = np.asanyarray(color_frame.get_data())
        rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
        return rgb

    # def get_image(self):
    #     frames = self.pipeline.wait_for_frames()
    #     color_frame = frames.get_color_frame()
    #     if not color_frame:
    #         return None
        
    #     bgr = np.asanyarray(color_frame.get_data())
    #     rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
        
    #     # Center crop to (180, 320) - height=180, width=320
    #     h, w = rgb.shape[:2]
    #     target_h, target_w = 180, 320
        
    #     # Calculate crop coordinates
    #     start_y = (h - target_h) // 2
    #     start_x = (w - target_w) // 2
    #     end_y = start_y + target_h
    #     end_x = start_x + target_w
        
    #     # Perform center crop
    #     rgb_cropped = rgb[start_y:end_y, start_x:end_x]
        
    #     return rgb_cropped


    def release(self):
        self.pipeline.stop()



# For FRE
# camera serial numbers
if LOCATION == "FRE":
    serial_left = "934222071783"
    serial_right = "925622070414"
elif LOCATION == "MPK":
    serial_left = "948522071060"
    serial_right = "838212074411"

# # initialize cameras
cam_left = RealSenseCamera(serial_left)
cam_right = RealSenseCamera(serial_right)


# inicialize zed mini wrist camera

In [ ]:
import pyzed.sl as sl
import numpy as np
import cv2

class ZEDMiniCamera:
    def __init__(self, resolution=sl.RESOLUTION.HD720, fps=30, depth=False):
        print("[INFO] Initializing ZED Mini camera")
        self.zed = sl.Camera()

        init_params = sl.InitParameters()
        init_params.camera_resolution = resolution
        init_params.camera_fps = fps
        init_params.depth_mode = sl.DEPTH_MODE.PERFORMANCE if depth else sl.DEPTH_MODE.NONE
        init_params.coordinate_units = sl.UNIT.MILLIMETER  # For depth, if used

        status = self.zed.open(init_params)
        if status != sl.ERROR_CODE.SUCCESS:
            raise RuntimeError(f"[ERROR] Failed to open ZED camera: {status}")
        print("[SUCCESS] ZED camera opened successfully")

        self.image = sl.Mat()

    def get_image(self):
        if self.zed.grab() == sl.ERROR_CODE.SUCCESS:
            self.zed.retrieve_image(self.image, sl.VIEW.LEFT)  # LEFT image (color)
            bgr_image = self.image.get_data()
            rgb_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)
            return rgb_image
        else:
            print("[WARN] Failed to grab frame from ZED")
            return None


    # def get_image(self):
    #     if self.zed.grab() == sl.ERROR_CODE.SUCCESS:
    #         self.zed.retrieve_image(self.image, sl.VIEW.LEFT)  # LEFT image (color)
    #         bgr_image = self.image.get_data()
    #         rgb_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)
            
    #         # Center crop to (180, 320) - height=180, width=320
    #         h, w = rgb_image.shape[:2]
    #         target_h, target_w = 180, 320
            
    #         # Calculate crop coordinates
    #         start_y = (h - target_h) // 2
    #         start_x = (w - target_w) // 2
    #         end_y = start_y + target_h
    #         end_x = start_x + target_w
            
    #         # Perform center crop
    #         rgb_cropped = rgb_image[start_y:end_y, start_x:end_x]
            
    #         return rgb_cropped
    #     else:
    #         print("[WARN] Failed to grab frame from ZED")
    #         return None

    def release(self):
        self.zed.close()
        print("[INFO] ZED camera released")


zed_cam = ZEDMiniCamera(resolution=sl.RESOLUTION.VGA, fps=30)

## get current images

In [ ]:
from openpi_client import image_tools

img_left = image_tools.resize_with_pad(
                        cam_left.get_image(), 224, 224
                    )
# img_wrist = image_tools.resize_with_pad(
#                         cam_wrist.get_image(), 224, 224
#                     )

img_wrist = image_tools.resize_with_pad(
                        zed_cam.get_image(), 224, 224
                    )

img_right = image_tools.resize_with_pad(
                        cam_right.get_image(), 224, 224
                    )

# combine the two images
combined = np.hstack((img_left, img_wrist, img_right))  # shape (224, 672, 3)

# display the combined image
plt.figure(figsize=(8, 4))
plt.imshow(combined)
plt.title("Left + Wrist + Right (RGB 224x224)")
plt.axis('off')
plt.show()





# release cameras

In [ ]:
cam_left.release()
cam_right.release()
zed_cam.release()

# openpi inference client

In [ ]:
# vla_policy_client
import Pyro5.api
import numpy as np
import time
from openpi_client import image_tools

ns = Pyro5.api.locate_ns()  # Locate the name server
uri = ns.lookup("pi0_controller")  # Use the name server to look up the URI
controller = Pyro5.api.Proxy(uri)

# prompt
prompt = "pick up the red bowl"  # <-- change prompt

MAX_STEPS = 50000  # maximum number of steps to run
video_buffer = []  # buffer to store video frames

# dummy action
action = np.zeros((10,8), dtype=np.float32)
action_list = action.tolist()  # convert to list for sending

for step in range(MAX_STEPS):
    start = time.time()
    print(f"\n=== Step {step} ===")
    data_to_send = {
        "action": action_list,
        "step": step
    }
    start1 = time.time()
    print(f"Sending data to controller: {data_to_send}")
    obs = controller.step(data_to_send)
    end1 = time.time()
    print(f"Controller step took {end1 - start1:.2f} seconds")

    img_left = image_tools.resize_with_pad(
                            cam_left.get_image(), 224, 224
                        )
    img_wrist = image_tools.resize_with_pad(
                            zed_cam.get_image(), 224, 224
                        )
    img_right = image_tools.resize_with_pad(
                            cam_right.get_image(), 224, 224
                        )
    
    # save the images
    combined = np.hstack([img_left, img_right, img_wrist])
    video_buffer.append(combined)

    observation = {
        "observation/exterior_image_1_left": img_left,
        # "observation/exterior_image_1_left": np.zeros_like(img_left),  # dummy image for left
        "observation/wrist_image_left": img_wrist,
        # "observation/wrist_image_left": np.zeros_like(img_wrist),  # dummy image for wrist
        "observation/joint_position": np.array(obs["robot_state"], dtype=np.float32),
        "observation/gripper_position": np.array([obs["gripper_state"]], dtype=np.float32),  
        # "observation/joint_position": np.zeros_like(obs["robot_state"], dtype=np.float32),
        # "observation/gripper_position": np.zeros_like([obs["gripper_state"]], dtype=np.float32),  
        "prompt": prompt,
    }

    result = policy.infer(observation)
    action_list = result["actions"].tolist()
    end2 = time.time()
    print(f"Inference took {end2 - end1:.2f} seconds")
    print(f"Action at step {step}:", action_list[0])
    end = time.time()
    print(f"Step {step} took {end - start:.2f} seconds")


## save to gif

In [ ]:
import imageio
import cv2


gif_path = "pi0_droid_drawer_1.gif"
imageio.mimsave(gif_path, video_buffer, duration=0.1)
print(f"GIF saved:{gif_path}")

In [ ]:
from IPython.display import Image
Image(filename="pi0_droid_drawer_1.gif")